In [4]:

import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
import json

In [6]:
import json
dataList = []
sentences = []
labels = []
def loadDataset(filename):
  d=[]
  with open(filename, 'r') as f:
      datastore = json.load(f)                  
  for item in datastore:
    sentence = item['data']
    label = item['is_sensitive']
    d.append([sentence,label])
  return d
# Loading both sensitive and non-sensitive dataset
sen=loadDataset("SensitiveDataset.json")
nonsen=loadDataset("NonSensitiveDatasetnew.json")
sen=pd.DataFrame(data=sen,columns=['sentences','labels'])
nonsen=pd.DataFrame(data=nonsen,columns=['sentences','labels'])

In [7]:
combined = pd.concat([sen, nonsen])
df = combined.sample(frac=1).reset_index(drop=True)
df.head()

,sentences,labels
0,"Colorado, 1865: A gold assayer is killed and L...",0
1,"kirsten, gaara, kirsten@comcast.net, 28...",1
2,"annabal, qazwsx, annabal@sfr.fr, 785751...",1
3,"Taj Badalandabad (Kal Penn), the personal assi...",0
4,Bernie LaPlante (Dustin Hoffman) is a pickpock...,0


In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

tokenized_data = [word_tokenize(sentence) for sentence in df['sentences']]
stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
punctuations.remove('@')
stop_words.update(punctuations)
stemmer = PorterStemmer()
filtered_data = [[stemmer.stem(word) for word in sentence if word.lower() not in stop_words] for sentence in tokenized_data]
l=[]
for idx, sentence in enumerate(filtered_data):
    l.append(' '.join(sentence))         
df['sentences'] = l

In [9]:
df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

(25261, 2) (6315, 2)


In [10]:
y_train = df_train['labels'].values
y_test = df_test['labels'].values
y_test.shape

(6315,)

In [11]:
X_train = df_train['sentences'].values
X_test = df_test['sentences'].values

In [12]:
#import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word

In [13]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
print(X_train_seq[:5])
print(df_train.iloc[0,:])
for el in X_train_seq[0]:
    print(word_dict[el], end=' ')

[[624, 446, 1272, 3470, 20125, 8, 110, 2450, 70, 674, 247, 603, 431, 71, 269, 132, 123, 643, 1046, 252, 256, 22, 10883, 2454, 15, 438, 643, 2, 348, 110, 285, 3909, 246, 3619, 4, 20125, 3470, 149, 1074, 3336, 19, 1193, 3470, 52, 55, 954, 643, 77, 2, 22, 43, 305, 166, 3470, 1907, 429, 6067, 10883, 11, 492, 315, 2454, 1044, 1440, 2569, 1044, 185, 1440, 613, 1234, 2569, 8747, 2454, 169, 227, 492, 1044, 1440, 22, 2454, 102, 2721, 4020, 10883, 643, 232, 643, 348, 110, 285, 52, 20, 252, 256, 14, 3619, 149, 3470, 337, 5557, 1074, 17768, 1261, 2454, 1943, 117, 18, 20125, 11830, 675, 643, 78, 12608, 915, 192, 913, 595, 35, 3197, 914, 65, 1022, 51, 252, 2, 35, 19, 992, 18, 3602, 252, 2, 3299, 242, 1373, 120, 643, 19, 1109, 20125, 349, 252, 992, 252, 264, 643, 9, 1220, 2515, 211, 643, 14, 3619, 1598, 3470, 19, 6690, 7894, 1981, 243, 2454, 19, 133, 32, 675, 489, 8, 1234, 52, 55, 2569, 51, 772, 914, 327, 1981, 348, 7, 55, 1044, 1440, 136, 299, 10883, 1193, 32, 53, 1440, 2212, 92, 11503, 243, 261, 52

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')
X_train_pad[:5]
X_train_pad.shape[0]
#get max number in X_train_pad and X_test_pad
max_num=max([max(seq) for seq in X_train_pad])
max_num

82074

In [15]:
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
#implement the SVM model
clf = svm.SVC(kernel='linear')
clf.fit(X_train_pad, y_train)
y_pred = clf.predict(X_test_pad)
#classification report
print(classification_report(y_test, y_pred))
#accuracy score
print('Linear',accuracy_score(y_test, y_pred))
"""
#Try different kernels
clf = svm.SVC(kernel='rbf')
clf.fit(X_train_pad, y_train)
y_pred = clf.predict(X_test_pad)
print(classification_report(y_test, y_pred))
print('RBF',accuracy_score(y_test, y_pred))

clf = svm.SVC(kernel='poly')
clf.fit(X_train_pad, y_train)
y_pred = clf.predict(X_test_pad)
print(classification_report(y_test, y_pred))
print('Poly',accuracy_score(y_test, y_pred))

clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train_pad, y_train)
y_pred = clf.predict(X_test_pad)
print(classification_report(y_test, y_pred))
print('Sigmoid',accuracy_score(y_test, y_pred))  
"""

: 

: 